In [24]:
import cv2
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
sys.path.insert(1, 'C:/Users/chuck/OneDrive/Desktop/Honors/solarEL/solarel')
from image_processor import split_module_to_cells
import time

%matplotlib qt


In [25]:
def euclidean_distance(p1, p2):
        return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)


def average_point(point1, point2):
    avg_x = (point1[0] + point2[0]) / 2
    avg_y = (point1[1] + point2[1]) / 2
    return (avg_x, avg_y)


def merge_points_within_distance(points, distance_threshold):
    # Function to calculate Euclidean distance between two points
    
    # List to store merged points
    merged_points = []
    
    # Iterate through x and y lists
    for point in points:
        
        should_merge = False
        
        # Check against points already merged
        for i, merge_point in enumerate(merged_points):
            if euclidean_distance(point, merge_point) <= distance_threshold:
                should_merge = True
                break
        
        # If not merged, add the point to the merged lists
        if not should_merge:
            merged_points.append(point)
            continue
        
        new_point = average_point(point, merge_point)
        
        merged_points.append(new_point)
        merged_points.pop(i)
    
    return merged_points

def pixel_distances(points_1, points_2, threshold):
    
    distances = []

    # Iterate through x and y lists
    for point_1 in points_1:
        
        close_enough = False

        # Check against points already merged
        for point_2 in points_2:
            if euclidean_distance(point_1, point_2) <= threshold:
                close_enough = True
                break
        
        # If not merged, add the point to the merged lists
        if not close_enough:
            continue
        
        error = euclidean_distance(point_1, point_2)
        distances.append(error)
    
    return distances


def test_accuracy(image_list, csv_list, model_path_cells, model_path_module, toml_path, show=False):
    times = [0]*10
    errors = list()
    for i, image_name in enumerate(image_list):
        #read image
        image = cv2.imread(image_name)
        
        # read true values from csv
        data = pd.read_csv(csv_list[i])
        x_values = data['x']
        y_values = data['y']
        float_points = zip(x_values, y_values)
        int_points = [(int(x),int(y)) for (x,y) in float_points]

        # copy image for display purposes
        image_cp = np.copy(image)
        if show:
            plt.figure(i)
            for x, y in zip(x_values, y_values):
                point_color = (0, 0, 255)  # Red color
                point_radius = 10
                cv2.circle(image_cp, (int(x), int(y)), point_radius, point_color, 3)
        
        # time the cell splitting algorithm
        start = time.time()
        points = split_module_to_cells(model_path_module, model_path_cells, image, toml_path, coords=True, barrel_fix=False, module_crop=False, show=True)
        end = time.time()
        x = end-start
        times[i] = x

        # Merge all corner points that are close to mathc the format of the csv
        x_predicted = []
        y_predicted = []
        for point_array in points:
            x_predicted.extend(point_array[:, 0])
            y_predicted.extend(point_array[:, 1])
        x_predicted_int = [int(item) for item in x_predicted]
        y_predicted_int = [int(item) for item in y_predicted]
        original_points = zip(x_predicted_int, y_predicted_int)
        predicted_points = merge_points_within_distance(original_points, 20)
        predicted_points_int = [(int(x), int(y)) for x, y in predicted_points]

        # Loop through points and draw them on the image
        if show:
            for point in predicted_points_int:
                image_cp = cv2.drawMarker(image_cp, point, (255,0,0), cv2.MARKER_TILTED_CROSS, 10, 3)
            
            plt.imshow(image_cp) #, interpolation='none'
            plt.show()

        errors.append(pixel_distances(int_points, predicted_points_int, 20))
    return errors, times

In [26]:
model_path_cells = (
    "C:/Users/chuck/OneDrive/Desktop/Honors/models/resnet_backbone_512.hdf5"
)
model_path_module = (
    "C:/Users/chuck/OneDrive/Desktop/Honors/models/resnet_PV_module_256.hdf5"
)
toml_path = "C:/Users/chuck/OneDrive/Desktop/Honors/solarEL/solarel/configs/camera_config.toml"

image_list = []
csv_list = []
for i in range(10):
    image_list.append(f"images/{i}.jpg")
    csv_list.append(f"csv_points/{i}.csv")

In [27]:
errors, times = test_accuracy(image_list, csv_list, model_path_cells, model_path_module, toml_path, show=False)

1/1 [==============================] - 0s 360ms/step
(4260, 2224)
(4257, 2222)
1/1 [==============================] - 0s 456ms/step
(4160, 2172)
(4157, 2171)
1/1 [==============================] - 0s 345ms/step
(4124, 2156)
(4123, 2155)
1/1 [==============================] - 0s 360ms/step
(4108, 2156)
(4107, 2155)
1/1 [==============================] - 0s 386ms/step
(4096, 2160)
(4093, 2157)
1/1 [==============================] - 0s 404ms/step
(4176, 2208)
(4173, 2205)
1/1 [==============================] - 0s 406ms/step
(4008, 2472)
(4005, 2471)
1/1 [==============================] - 0s 387ms/step
(4088, 2140)
(4088, 2140)
1/1 [==============================] - 0s 414ms/step
(4192, 2188)
(4191, 2188)
1/1 [==============================] - 0s 434ms/step
(4200, 2200)
(4200, 2200)


In [28]:
print(np.mean(errors))
print(np.mean(times))
#[8.514387656875497, 7.864104147913357, 7.691576716967159, 9.515913388021133, 10.047067421858237, 10.197183017655139, 11.261587845710284, 9.812369180008314, 8.33495891171388, 12.224940455020022]

[1.25299641 1.58113883 1.1        1.48660687 1.41421356 1.80277564
 0.41231056 1.20415946 0.1        1.11803399 0.2        1.08166538
 1.92353841 0.64031242 1.20415946 0.60827625 0.85440037 0.3
 0.85440037 1.5132746  0.41231056 0.90553851 0.2        0.82462113
 0.1        1.1        1.82482876 0.6        0.70710678 0.58309519
 0.78102497 0.5        1.08166538 1.99248588 0.36055513 0.41231056
 0.72801099 0.67082039 0.50990195 1.30384048 1.84390889 0.28284271
 0.42426407 0.80622577 0.78102497 0.6        1.25299641 1.82482876
 0.82462113 0.63245553 0.60827625 0.60827625 0.         1.04403065
 0.58309519 0.4472136  0.72111026 0.56568542 0.76157731 0.92195445
 1.55241747 0.7        0.70710678 0.60827625 0.4        0.42426407
 0.78102497 1.50332964 1.50332964 1.16619038 0.5        0.5
 0.4        0.31622777 1.20415946 1.23693169 0.78102497 0.58309519
 0.2        0.80622577 0.63245553 1.22065556 0.36055513 0.5
 0.89442719 0.9        1.36014705 1.4        1.7        1.70293864
 1.42126704 1.20

c:\Python\Python39\lib\site-packages\numpy\core\_methods.py:163: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)


In [29]:
np.shape(errors[0])

(88,)

In [36]:
errors_module_1 = errors[0] + errors[1] + errors[2] + errors[5] + errors[9]
errors_module_2 = errors[3] + errors[8]
errors_module_3 = errors[4] + errors[7]
errors_module_4 = errors[6]
overall = []
[overall.extend(error) for error in errors]

[12.529964086141668, 15.811388300841896, 11.0, 14.866068747318506, 14.142135623730951, 18.027756377319946, 4.123105625617661, 12.041594578792296, 1.0, 11.180339887498949, 2.0, 10.816653826391969, 19.235384061671343, 6.4031242374328485, 12.041594578792296, 6.082762530298219, 8.54400374531753, 3.0, 8.54400374531753, 15.132745950421556, 4.123105625617661, 9.055385138137417, 2.0, 8.246211251235321, 1.0, 11.0, 18.24828759089466, 6.0, 7.0710678118654755, 5.830951894845301, 7.810249675906654, 5.0, 10.816653826391969, 19.924858845171276, 3.605551275463989, 4.123105625617661, 7.280109889280518, 6.708203932499369, 5.0990195135927845, 13.038404810405298, 18.439088914585774, 2.8284271247461903, 4.242640687119285, 8.06225774829855, 7.810249675906654, 6.0, 12.529964086141668, 18.24828759089466, 8.246211251235321, 6.324555320336759, 6.082762530298219, 6.082762530298219, 0.0, 10.44030650891055, 5.830951894845301, 4.47213595499958, 7.211102550927978, 5.656854249492381, 7.615773105863909, 9.219544457292

In [76]:
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

In [79]:
fig = plt.figure()
ax = fig.add_subplot(111)
data = [errors_module_1, errors_module_2, errors_module_3, errors_module_4, overall]
box_colour = (0/255, 169/255, 206/255)

bp = ax.boxplot(data,patch_artist=True)

# Customize the boxplot elements
for box in bp['boxes']:
    box.set(color='black', linewidth=1)
    box.set(facecolor=box_colour)  # Set the fill color for the box

for flier in bp['fliers']:
    flier.set(marker='o', markersize=8, markerfacecolor=box_colour, markeredgecolor='black')

for median in bp['medians']:
    median.set(color='black', linewidth=2)

x_labels = ["Module\nType 1", "Module\nType 2", "Module\nType 3", "Module\nType 4", "Overall"]
font1 = {'family': 'georgia', 'size': 20}
plt.xticks(range(1, len(x_labels) + 1), x_labels, rotation=45, fontdict=font1)
plt.tick_params(axis='y', labelsize=18)

ax.set_facecolor('white')

font2 = {'family': 'georgia', 'size': 24}
ax.set_ylabel("Average Pixel Error", fontdict=font2)
ax.set_xlabel("Module Types", fontdict=font2)
plt.show()